In [1]:
import threading
import socket
import struct
import os
import sys

In [3]:
def socket_service():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        s.bind(('192.168.11.250', 9001))
        s.listen(10)
    except socket.error as msg:
        print(msg)
        sys.exit(1)

    print('等待连接...')
    clients = {}  # 保存连接的字典

    while True:
        conn, addr = s.accept()
        ip, port = addr
        # 假设你已经知道哪个IP是发送者，哪个是接收者
        if ip == '192.168.11.101':  # 发送者的IP
            role = 'sender'
        elif ip == '192.168.11.102':  # 接收者的IP
            role = 'receiver'
        else:
            print(f"未授权的客户端尝试连接: {ip}")
            conn.close()
            continue

        clients[role] = conn
        threading.Thread(target=deal_data, args=(conn, addr, role, clients)).start()

In [4]:
def deal_data(conn, addr, role, clients):
    print(f'客户端地址为： {addr}, 角色为： {role}')
    try:
        if role == 'sender':
            receive_file_from_sender(conn, addr, clients)
        elif role == 'receiver':
            print('接收者准备接收文件')
    except Exception as e:
        print(f"处理过程中出现异常: {e}")
    finally:
        conn.close()
        print(f'{role}连接已关闭。')

In [5]:
def receive_file_from_sender(conn, addr, clients):
    fileinfo_size = struct.calcsize('128sl')
    buf = conn.recv(fileinfo_size)
    if buf:
        filename, filesize = struct.unpack('128sl', buf)
        fn = filename.strip(b'\00').decode()
        print(f'数据名为 {fn}, 数据大小为 {filesize}')
        recvd_size = 0
        with open(f'/path/to/store/{fn}', 'wb') as fp:
            print('开始接收客户端数据...')
            while recvd_size < filesize:
                data = conn.recv(1024)
                recvd_size += len(data)
                fp.write(data)
            print('客户端数据接收完成...')

        if 'receiver' in clients:
            send_file_to_receiver(fn, clients['receiver'])

In [6]:
def send_file_to_receiver(filename, receiver_conn):
    path = f'/path/to/store/{filename}'
    with open(path, 'rb') as fp:
        fileinfo = struct.pack('128sl', filename.encode('utf-8'), os.stat(path).st_size)
        receiver_conn.send(fileinfo)
        while True:
            data = fp.read(1024)
            if not data:
                break
            receiver_conn.send(data)
        print(f'文件{filename}已发送到接收者')

In [ ]:
socket_service()